In [1]:
import torch
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import sys

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def generate(input_str):
    # change your default input according to your model

    # eeve model (yanolja)
    inputs = tokenizer(f"{input_str}\n\n### Response: ", return_tensors="pt").to("cuda")
    # Mistral & llama 2 model
    # inputs = tokenizer('[INST]'+input_str+'[/INST]', return_tensors="pt").to("cuda")

    # inputs = tokenizer(input_str, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            pad_token_id=tokenizer.pad_token_id,
            input_ids=inputs["input_ids"],
            max_new_tokens=1,
            top_p=0.6,
            top_k=1,
            temperature=0.9,
            repetition_penalty=1.0,
            num_beams=1,
            early_stopping=False
        )

        while int(outputs[0][-1].detach().cpu().numpy()) != tokenizer.eos_token_id:
            # yield after [/INST] token, if not using [/INST] token, remove split("[/INST]")[1]
            yield tokenizer.batch_decode(outputs.detach().cpu().numpy())[0]#.split("[/INST]")[1]
            outputs = model.generate(
                pad_token_id=tokenizer.pad_token_id,
                input_ids=outputs,
                max_new_tokens=1,
                top_p=0.6,
                top_k=1,
                temperature=0.9,
                repetition_penalty=1.0,
                num_beams=1,
                early_stopping=False
            )


In [3]:
model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = 'auto',
    trust_remote_code = True,
    # use_cache = True,
    # ignore_mismatched_sizes=False,
    # low_cpu_mem_usage=True,
    # load_in_8bit = True, 
    ).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    trust_remote_code=True,
)

model.eval()

print("Model loaded successfully.")

Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.74it/s]
tokenizer_config.json: 100%|██████████| 1.86k/1.86k [00:00<00:00, 8.45MB/s]
tokenizer.json: 100%|██████████| 2.18M/2.18M [00:00<00:00, 2.33MB/s]
special_tokens_map.json: 100%|██████████| 557/557 [00:00<00:00, 2.36MB/s]

Model loaded successfully.


In [7]:
demo = gr.Interface(
    fn=generate,
    inputs=gr.inputs.Textbox(lines=20, label="Input Text"),
    outputs=gr.outputs.Textbox(label="Generated Text"),
    title="LIGHT-INFERENCE",
    description=f"LLM RAG chatbot demo",
)

demo.queue()#max_size=2)
demo.launch(share=True)

/tmp/ipykernel_3876815/879999510.py:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=20, label="Input Text"),
/tmp/ipykernel_3876815/879999510.py:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=20, label="Input Text"),
/tmp/ipykernel_3876815/879999510.py:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=20, label="Input Text"),
/tmp/ipykernel_3876815/879999510.py:4: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(label="Generated Text"),


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://5b61f05f1f0bab4112.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
